In [2]:
import pandas as pd
import numpy as np
import datetime
from indicators import *
from candle_creation import *
import os
import operator

In [3]:
data_dir = "C:\\Users\\TEMP002\\Downloads\\Complete Dataset\\CSV\\"
market_start_time = datetime.time(9,15,0)
market_end_time = datetime.time(15,29,0)

index = "NIFTY"
data_type = "SPOT"
start_year = 2016
end_year = 2022

spot_files = []
for year in range(start_year,end_year+1):
    year_dir = data_dir+index+'\\'+str(year)+'\\'
    for path_i in os.listdir(year_dir):
        month_dir = os.path.join(year_dir,path_i)
        for path_j in os.listdir(month_dir):
            if path_j == data_type:
                spot_dir = os.path.join(month_dir,path_j)
                for file_name in os.listdir(spot_dir):
                       spot_files.append(os.path.join(spot_dir,file_name))

In [4]:
all_data = pd.DataFrame()
for file in spot_files:
    df = pd.read_csv(file,index_col=0,parse_dates=True,usecols=['Date','Time','Open','High','Low','Close'])
    df['Time'] = df['Time'].apply(lambda x:(pd.to_datetime(x)-pd.to_timedelta("59S")).time())
    time_vals = df['Time'].between(market_start_time,market_end_time)
    df=df[time_vals]
    #df['time_stamp'] = df.apply(lambda x: x.name.replace(hour = x.Time.hour, minute = x.Time.minute),axis=1)
    all_data = pd.concat([all_data,df])

In [6]:
all_data['time_stamp'] = all_data.apply(lambda x: x.name.replace(hour = x.Time.hour, minute = x.Time.minute),axis=1)
all_data.sort_values(by='time_stamp',inplace=True)

In [7]:
data = get_Discrete_candles(all_data,freq="240T")

In [9]:
data.head(5)

,Time,Open,High,Low,Close,time_stamp
Date,,,,,,
2016-10-20,09:15:00,8691.85,8727.00,8678.30,8696.30,2016-10-20 09:15:00
2016-10-20,13:15:00,8696.85,8710.70,8687.45,8700.20,2016-10-20 13:15:00
2016-10-21,09:15:00,8701.95,8703.45,8652.05,8672.30,2016-10-21 09:15:00
2016-10-21,13:15:00,8672.85,8698.80,8654.25,8691.15,2016-10-21 13:15:00
2016-10-24,09:15:00,8707.40,8724.75,8684.15,8721.90,2016-10-24 09:15:00


In [8]:
all_data.head(5)

,Time,Open,High,Low,Close,time_stamp
Date,,,,,,
2016-10-20,09:15:00,8691.85,8696.35,8687.75,8688.90,2016-10-20 09:15:00
2016-10-20,09:16:00,8689.50,8694.00,8688.90,8694.00,2016-10-20 09:16:00
2016-10-20,09:17:00,8693.85,8695.15,8692.80,8694.65,2016-10-20 09:17:00
2016-10-20,09:18:00,8694.15,8697.65,8694.15,8697.00,2016-10-20 09:18:00
2016-10-20,09:19:00,8696.80,8700.40,8696.80,8699.65,2016-10-20 09:19:00


In [6]:
all_data['time_stamp'] = all_data.apply(lambda x: x.name.replace(hour = x.Time.hour, minute = x.Time.minute),axis=1)
all_data.sort_values(by='time_stamp',inplace=True)

In [10]:
data = get_Discrete_candles(all_data,freq="240T")

In [11]:
data = fractals(data)
data.set_index('time_stamp',inplace=True)
data.drop(columns=['Time'],axis=1,inplace=True)

In [13]:
filter_1 = data['is_high'] == 1
filter_2 = data['is_low'] == 1
data = data[filter_1 | filter_2]

data.drop(columns = ['Open','HF_appeared','LF_appeared'],axis=1,inplace=True)

In [61]:
data = data[['High', 'Low', 'Close', 'is_high', 'is_low', 'fractal_time']]

In [14]:
data.head()

,High,Low,Close,is_high,is_low,fractal_time
time_stamp,,,,,,
2016-10-21 09:15:00,8703.45,8652.05,8672.30,0,1,09:15:00
2016-10-24 13:15:00,8736.95,8701.60,8711.65,1,0,13:15:00
2016-10-27 09:15:00,8609.90,8550.25,8589.55,0,1,09:15:00
2016-11-01 13:15:00,8669.60,8618.60,8624.50,1,0,13:15:00
2016-11-04 13:15:00,8458.85,8400.25,8435.65,0,1,13:15:00


## PREV_AVG AND FRACTAL VALUE

In [15]:
def get_first_n_previous_indices(data,win_len):
    prev_n_red = []
    prev_n_green = []
    
    for index,row in data.iterrows():
        if row.is_high and len(prev_n_green)<win_len:  #Checking if we got previous_n_highs so we can start computation
            prev_n_green.append(index)

        if row.is_low and len(prev_n_red)<win_len:   #Checking if we got previous_n_lows
            prev_n_red.append(index)

        if len(prev_n_green) == win_len and len(prev_n_red) == win_len: #Start index i.es we
            break
    
    return prev_n_red,prev_n_green

In [16]:
def get_n_trailing_averages_new(data,win_len=5):
    
    prev_n_red, prev_n_green = get_first_n_previous_indices(data,win_len)

    if prev_n_red[-1]<prev_n_green[-1]:  ## Start index of calculation
        current_index = prev_n_red[-1]
    else:
        current_index = prev_n_green[-1]
    
    start_row_num = data.index.get_loc(prev_n_green[-1]) + 1
    red_start_index = data.iloc[start_row_num].name

    start_row_num = data.index.get_loc(prev_n_red[-1]) + 1
    green_start_index = data.iloc[start_row_num].name
    
    prev_n_op_avg = pd.Series(dtype="float64", index=data.index) #Initializing a Pandas Series to store trailing avg
    
    prev_n_op_red_avg = pd.Series(dtype="float64", index=data.index)
    prev_n_op_green_avg = pd.Series(dtype="float64", index=data.index)

    for index,row in data.loc[green_start_index:].iterrows():
        if row.is_high:
            prev_n_low_avg = data.loc[prev_n_red]['Low'].mean()
            prev_n_op_red_avg.loc[index] = prev_n_low_avg

        if row.is_low:
            prev_n_red.pop(0)
            prev_n_red.append(row.name)
            
    for index,row in data.loc[red_start_index:].iterrows():
        if row.is_low:
            prev_n_high_avg = data.loc[prev_n_green]['High'].mean()
            prev_n_op_green_avg.loc[index] = prev_n_high_avg

        if row.is_high:
            prev_n_green.pop(0)
            prev_n_green.append(row.name)
    
            
    data = data.assign(prev_n_op_red_avg = prev_n_op_red_avg)
    data = data.assign(prev_n_op_green_avg = prev_n_op_green_avg)
    
    return data


In [12]:
data = get_n_trailing_averages_new(data,5)

In [17]:
def get_fractal_val_green(is_high,high,low):
    if is_high:
        return high
    else:
        return np.NaN

def get_fractal_val_red(is_low,high,low):
    if is_low:
        return low
    else:
        return np.NaN

In [14]:
data['fractal_val_green'] = data.apply(lambda row : get_fractal_val_green(row.is_high,row.High,row.Low),axis=1)
data['fractal_val_red'] = data.apply(lambda row : get_fractal_val_red(row.is_low,row.High,row.Low),axis=1)


## ABS_DIFF AND PERCENT_GROWTH

In [18]:
def compute_abs_diff(is_low,is_high,prev_n_op_red_avg,prev_n_op_green_avg,fractal_val_green,fractal_val_red):
    
    if is_high ==1 and is_low==0:
        return fractal_val_green - prev_n_op_red_avg

    elif is_high == 0 and is_low == 1:
        return fractal_val_red - prev_n_op_green_avg

    elif is_high == 1 and is_low == 1:
        return np.NaN


def compute_percent_growth(is_low,is_high,prev_n_op_red_avg,prev_n_op_green_avg,fractal_val_green,fractal_val_red):
    
    if is_high ==1 and is_low==0:
        return (fractal_val_green/prev_n_op_red_avg -1)*100
    elif is_high == 0 and is_low == 1:
        return (fractal_val_red/prev_n_op_green_avg -1)*100
    elif is_high == 1 and is_low == 1:
            return np.NaN

In [16]:
data['abs_diff'] = data.apply(lambda row: compute_abs_diff(row.is_low,row.is_high,row.prev_n_op_red_avg,row.prev_n_op_green_avg,row.fractal_val_green,row.fractal_val_red),axis=1)
data['%_growth'] = data.apply(lambda row: compute_percent_growth(row.is_low,row.is_high,row.prev_n_op_red_avg,row.prev_n_op_green_avg,row.fractal_val_green,row.fractal_val_red),axis=1)


In [17]:
neutral_indices = data[(data['is_high']==1) & (data['is_low']==1)].index

green_start = data['prev_n_op_red_avg'].first_valid_index()
red_start = data['prev_n_op_green_avg'].first_valid_index()

if green_start<red_start:
    start_index = green_start
else:
    start_index = red_start


neutral_index_is_high = pd.Series(dtype="float64",index=data.index)

for index in neutral_indices:
    prev_total_data = data.loc[start_index:index]
    prev_normal_data = prev_total_data[~((prev_total_data['is_high']==1) & (prev_total_data['is_low']==1))]

    if not prev_normal_data.empty:
        prev_last_normal_rec = prev_normal_data.iloc[-1]

        if prev_last_normal_rec.is_high:
            data.at[index,'abs_diff'] = data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg']
            data.at[index,'%_growth'] = (data.loc[index]['fractal_val_green']/data.loc[index]['prev_n_op_red_avg'] - 1)*100
            neutral_index_is_high[index] = 1
        else:
            data.at[index,'abs_diff'] = data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg']
            data.at[index,'%_growth'] = (data.loc[index]['fractal_val_red']/data.loc[index]['prev_n_op_green_avg'] - 1)*100
            neutral_index_is_high[index] = 0
    
    
    else:
        if (data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg']) < 0:
            data.at[index,'abs_diff'] = (data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg'])
            data.at[index,'%_growth'] = (data.loc[index]['fractal_val_green']/data.loc[index]['prev_n_op_red_avg'] - 1)*100
            neutral_index_is_high[index] = 1

        elif (data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg']) > 0:
            data.at[index,'abs_diff'] = (data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg'])
            data.at[index,'%_growth'] = (data.loc[index]['fractal_val_red']/data.loc[index]['prev_n_op_green_avg'] - 1)*100
            neutral_index_is_high[index] = 0
        else:
            data.at[index,'abs_diff'] = np.NaN

data = data.assign(neutral_index_is_high = neutral_index_is_high)


## LEVEL & LEVEL_CHANGE

In [19]:
def change_label(label):
    if label == 'A':
        return 'B'
    else:
        return 'A'

In [20]:
def generate_levels(data,base_col):
    
    level = pd.Series(dtype='str',index=data.index)
    current_label = 'A'                                 #Assigning label A to the first type of fractal that occurs
    op_map={'is_high':operator.lt,'is_low':operator.gt} #mapping the type of operation depending on the type of fractal
    
    for index,row in data.iterrows():
        if row.is_low ==1  and row.is_high==0 and  row.abs_diff>0:
            start_index = index
            label_map = {'A':'is_high','B':'is_low'}
            level[index] = 'A'
            break
        if row.is_high == 1  and row.is_low == 0 and  row.abs_diff<0:
            start_index = index
            label_map = {'A':'is_low', 'B':'is_high'}
            level[index] = 'A'
            break

        if row.is_high ==1 and row.is_low ==1 :
            if row['neutral_index_is_high'] == 0 and row.abs_diff>0:
                start_index = index
                label_map = {'A':'is_high','B':'is_low'}
                level[index] = 'A'
                break
            if row['neutral_index_is_high'] == 1 and row.abs_diff<0:
                start_index = index
                label_map = {'A':'is_low', 'B':'is_high'}
                level[index] = 'A'
                break

    temp = {'is_high':1,'is_low':0}
    neutral_map = {i:temp[label_map[i]] for i in label_map}



            
    start_loc = data.index.get_loc(start_index)+1
    start_index = data.index[start_loc]
    
    for index,row in data.loc[start_index:].iterrows():
        if row[label_map[current_label]] == 1:                          #We doing the check depending on the previous row level
            opposite_label = change_label(current_label)

            if row[label_map[opposite_label]] == 0:                 # Not a neutral fractal
                if op_map[label_map[current_label]](row[base_col],0):  # checking if the next oppsoite fractal meets the criteria 
                    current_label = change_label(current_label)          #Switching the current label to opposite
                    level[index] = current_label
                else:
                    level[index] = current_label

            else:                                               #Is as neutral fractal
                if row['neutral_index_is_high'] == neutral_map[current_label]:      #Checking for a neutral fractal if it should be computed or not
                    
                    if op_map[label_map[current_label]](row[base_col],0):  # checking if the next oppsoite fractal meets the criteria 
                        current_label = change_label(current_label)          #Switching the current label to opposite
                        level[index] = current_label
                    else:
                        level[index] = current_label

                else:
                    level[index] = current_label

        else:
            level[index] = current_label
            
    data = data.assign(level=level)
    return data,label_map,op_map


In [21]:
def classification_1(data,base_col,thresh,op_col = 'CL-1'):
    data.loc[data[base_col].between(float("-inf"),-1*thresh,'right'), op_col] = 'Declining'
    data.loc[data[base_col].between(-1*thresh,thresh,'both'),op_col] = 'Same'
    data.loc[data[base_col].between(thresh,float("inf"),'left'),op_col] = 'Increasing'
    return data

In [22]:
def bin_abs_diff(data,label_map,thresh=0.5):

    new_map = {'A':label_map['B'],'B':label_map['A']}  ##now we are tracking exactly as per labels
    temp = {1:'is_high',0:'is_low'}
    reverse_map =  {new_map[i]:i for i in new_map} 
    neutral_index_map = {reverse_map[temp[i]]:i  for i in temp}

    data['level_change'] = data['level'].apply(lambda x : 1 if x=='A' else 0) 
    data['level_change'] = abs(data['level_change'].diff()) #Finding where cycle change happened
    data=classification_1(data,'%_growth',thresh)
    data['CL-1'] = data.apply(lambda row : 'First' if row['level_change'] else row['CL-1'],axis=1)

    data.loc[(data[label_map['A']]==1) & (data[label_map['B']]==0) & (data['level'] == 'A'),'CL-1'] = np.NaN
    data.loc[(data[label_map['B']]==1) & (data[label_map['A']]==0) & (data['level'] == 'B'),'CL-1'] = np.NaN

    data.loc[(data[label_map['A']]==1) & (data[label_map['B']]==1) & (data['level'] == 'B') & (data['neutral_index_is_high']==neutral_index_map['A']),'CL-1'] = np.NaN
    data.loc[(data[label_map['A']]==1) & (data[label_map['B']]==1) & (data['level'] == 'A') & (data['neutral_index_is_high']==neutral_index_map['B']),'CL-1'] = np.NaN

    data.at[data.index[0],'CL-1'] = np.NaN
    
    return data

In [22]:
data,label_map,op_map = generate_levels(data,'abs_diff')

In [23]:
data = bin_abs_diff(data,label_map,0.5)

## STEP-5 and CTA

In [23]:
def is_low_operation(fractal_val,CTA):
    op = (fractal_val/CTA) -1
    return op*100

def is_high_operation(fractal_val,CTA):
    op = (CTA/fractal_val) -1
    return op*100

In [24]:
def step_5_new(data,label_map):
    
    op_map={'is_high':is_high_operation,'is_low':is_low_operation}   #Storing the compute functions of High and Low Fractals
    new_map = {'A':label_map['B'],'B':label_map['A']}                # This mapping is just the opposite of 
    cycles_first_avg = pd.Series(dtype="float64",index=data.index)
    first_avg = np.NaN

    temp = {1:'is_high',0:'is_low'}
    reverse_map =  {new_map[i]:i for i in new_map} 
    neutral_index_map = {i:reverse_map[temp[i]] for i in temp}   ## To check whether step_5 has to to be calculated for 1,1 fractsl

    for index,row in data.iterrows():           ##LOCKING THE FIRST TRAILING AVERAGES OF A LEVEL
        if row['level_change'] ==1:
            if row.is_high == 1 and row.is_low == 0:                     #changed
                first_avg = row['prev_n_op_red_avg']
            elif row.is_low == 1 and row.is_high == 0:
                first_avg = row['prev_n_op_green_avg']
            elif row.is_high == 1 and row.is_low == 1:
                if row['neutral_index_is_high']:
                    first_avg = row['prev_n_op_red_avg']
                else:
                    first_avg = row['prev_n_op_green_avg']
                
            cycles_first_avg[index] = first_avg
        else:
            pass
        
    data = data.assign(CTA=cycles_first_avg)
    
    step_5 = pd.Series(dtype="float64",index=data.index)
    current_label = 'A'
    ## this variable tracks previous trailing avg for fractal
    prev_cycle_avg_map = {'A':data[data['level']=='A'].iloc[0]['CTA'] ,'B':data[data['level']=='B'].iloc[0]['CTA']}  #changed
    
    start_index = data[data['level_change']==1].iloc[0].name

    for index,row in data[start_index:].iterrows():

        if row.level_change == 1:                    ## This is for tracking which value to use for changing CTA used fro computation
            
            if row.is_high == 1 and row.is_low == 0:                           #changed
                prev_cycle_avg_map[row.level] = row['prev_n_op_red_avg']
            elif row.is_low == 1 and row.is_high == 0:
                prev_cycle_avg_map[row.level] = row['prev_n_op_green_avg']
            elif row.is_high == 1 and row.is_low == 1:
                if row['neutral_index_is_high']:
                    prev_cycle_avg_map[row.level] = row['prev_n_op_red_avg']
                else:
                    prev_cycle_avg_map[row.level] = row['prev_n_op_green_avg']
                      

        if row[new_map[current_label]]:

            compute = op_map[new_map[current_label]]
            if row.is_high == 1 and row.is_low == 0:
                fractal_val = row['fractal_val_green']
            elif row.is_high == 0 and row.is_low == 1:
                fractal_val = row['fractal_val_red']
            elif row.is_high == 1 and row.is_low ==1:
                fractal_val = np.NaN                         ## Initializing fractal_val to Nan So that if 1,1  doesnt match with current label we would get step5 NaN
                if current_label == neutral_index_map[row['neutral_index_is_high']]:
                    if row['neutral_index_is_high'] == 1:
                        fractal_val = row['fractal_val_green']
                    else:
                        fractal_val = row['fractal_val_red']
                
            val = compute(fractal_val,prev_cycle_avg_map[current_label])

            # if row.is_high ==1 and row.is_low ==1:
            #     print(index,'|',val,'|',current_label)

            if val<0:
                step_5[index] = val
                current_label = change_label(current_label)  
            else:
                step_5[index] = val
                
    data = data.assign(step_5 = step_5)
    return data,new_map


In [26]:
data,new_map = step_5_new(data,label_map)

## STEP-6

In [25]:
def step_6(data,new_map):
    
    step_6_map = {new_map[i]:i for i in new_map} 
    
    data.loc[(data['is_high']==1) & (data['is_low']==0) & (data['step_5']>0) ,'step_6'] = step_6_map['is_high']
    data.loc[(data['is_high']==1) & (data['is_low']==0) & (data['step_5']<0),'step_6'] = step_6_map['is_low']

    data.loc[(data['is_low']==1) & (data['is_high']==0) & (data['step_5']>0) ,'step_6'] = step_6_map['is_low']
    data.loc[(data['is_low']==1) & (data['is_high']==0) & (data['step_5']<0),'step_6'] = step_6_map['is_high']

    data.loc[(data['is_high']==1) & (data['is_low']==1) & (data['neutral_index_is_high']==1) & (data['step_5']>0),'step_6'] = step_6_map['is_high']
    data.loc[(data['is_high']==1) & (data['is_low']==1) & (data['neutral_index_is_high']==0) & (data['step_5']>0),'step_6'] = step_6_map['is_low']
    
    data.loc[(data['is_high']==1) & (data['is_low']==1) & (data['neutral_index_is_high']==1) & (data['step_5']<0),'step_6'] = step_6_map['is_low']
    data.loc[(data['is_high']==1) & (data['is_low']==1) & (data['neutral_index_is_high']==0) & (data['step_5']<0),'step_6'] = step_6_map['is_high']
   
    return data

In [28]:
data = step_6(data,new_map)

## STEP-7

In [26]:
def calc_step_7(data,new_map,step = 1):
    step_7 = pd.Series(dtype="float64",index=data.index)
    data = data.assign(step_7 = step_7)

    step_7_map = {'is_high':'prev_n_op_red_avg','is_low':'prev_n_op_green_avg'}
    
    only_A_firsts = data[(data['level_change']==1)&(data['level']=='A')]  #Fetching Only the StartRows of a partcular cycle 
    only_B_firsts = data[(data['level_change']==1)&(data['level']=='B')]
    
    only_A_firsts['shift'] = only_A_firsts[step_7_map[new_map['A']]].shift(step)   #Shifting trailing average
    only_B_firsts['shift'] = only_B_firsts[step_7_map[new_map['B']]].shift(step)
    
    only_A_firsts['step_7'] = only_A_firsts[step_7_map[new_map['A']]]/only_A_firsts['shift']  #Calc growrh ratio
    only_B_firsts['step_7'] = only_B_firsts[step_7_map[new_map['B']]]/only_B_firsts['shift']
    
    for index,row in only_A_firsts.iterrows():
        data.at[index,'step_7'] = row['step_7']
    
    for index,row in only_B_firsts.iterrows():
        data.at[index,'step_7'] = row['step_7']
        
    return data

In [30]:
data = calc_step_7(data,new_map)

C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_10508\1052621608.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_A_firsts['shift'] = only_A_firsts[step_7_map[new_map['A']]].shift(step)   #Shifting trailing average
C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_10508\1052621608.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_B_firsts['shift'] = only_B_firsts[step_7_map[new_map['B']]].shift(step)
C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_10508\1052621608.py:13: SettingWithCopyWar

# STEP -9 

In [27]:
def step_9(data,n=5,k=2):
    
    prev_n_green = []
    prev_n_red = []

    for index,row in data.iterrows():

        if row.is_high and len(prev_n_green)<n:
            prev_n_green.append(index)
        elif row.is_high and len(prev_n_green)==n:
            prev_n_green.pop(0)
            prev_n_green.append(index)

        if row.is_low and len(prev_n_red)<n:
            prev_n_red.append(index)
        elif row.is_low and len(prev_n_red)==n:
            prev_n_red.pop(0)
            prev_n_red.append(index)

        if len(prev_n_green) == n and len(prev_n_red)==n:
            break  
    
    
    if prev_n_red[-1]<prev_n_green[-1]:  ## Start index of calculation
        start_index = prev_n_green[-1]
        #prev_n_green.pop()   ## removing the latest index so loop can have continuity of logic from start row and we'll appen this index at the begining of loop

    elif prev_n_green[-1]<prev_n_red[-1]:
        start_index = prev_n_red[-1]
        #prev_n_red.pop

    elif prev_n_green[-1] == prev_n_red[-1]:
        start_index = prev_n_red[-1]
        
    n_green_avg = pd.Series(dtype="float64",index=data.index)
    n_red_avg = pd.Series(dtype="float64",index=data.index)

    for index,row in data[start_index:].iterrows():

        if index == start_index:        #Not popping the prev_red and prev_green indices 
            green_val = data.loc[prev_n_green]['High'].mean()
            red_val = data.loc[prev_n_red]['Low'].mean()

            n_green_avg.loc[index] = green_val
            n_red_avg.loc[index] = red_val

        else:

            if row.is_high :                    #When Green fractal comes pop the first element and add current index
                prev_n_green.pop(0)
                prev_n_green.append(index)

            if row.is_low:                      #When Red fractal comes pop the first element and add current index    
                prev_n_red.pop(0)
                prev_n_red.append(index)        #When both Red and Green comes we will any way have latest n using above logic


            n_green_avg.loc[index] = data.loc[prev_n_green]['High'].mean()
            n_red_avg.loc[index] = data.loc[prev_n_red]['Low'].mean()

    data['green_avg'] = n_green_avg
    data['red_avg'] = n_red_avg
    
    
    strided_green_avg = pd.Series(dtype="float64",index=data.index)
    strided_red_avg = pd.Series(dtype="float64",index=data.index)
    min_prev_candles = (k+1)*n

    for index,row in data.iterrows():
        sub_df = data.loc[:index]
        sub_df_high = sub_df[sub_df['is_high']==1]
        sub_df_low = sub_df[sub_df['is_low']==1]
        print(index)

        if sub_df_high.shape[0]>= min_prev_candles:

            last_index = sub_df_high.index[-1]
            row_num = sub_df_high.index.get_loc(last_index)

            end_index = sub_df_high.index[row_num-n*k]
            start_index = sub_df_high.index[row_num-n*(k+1)+1]

            prev_strided_green_avg = sub_df_high.loc[start_index:end_index]['High'].mean()  

            strided_green_avg.loc[index] = prev_strided_green_avg

        if sub_df_low.shape[0]>=min_prev_candles:

            last_index = sub_df_low.index[-1]
            row_num = sub_df_low.index.get_loc(last_index)

            end_index = sub_df_low.index[row_num-n*k]
            start_index = sub_df_low.index[row_num-n*(k+1)+1]

            prev_strided_red_avg = sub_df_low.loc[start_index:end_index]['Low'].mean()

            strided_red_avg.loc[index] = prev_strided_red_avg
            

    data['green_op'] = strided_green_avg
    data['red_op'] = strided_red_avg
        
    data['avg_green_growth'] = data['green_avg']/data['green_op']-1
    data['avg_red_growth'] = data['red_avg']/data['red_op']-1
    
    return data


In [41]:
data = step_9(data,10,2)


2016-10-20 09:51:00
2016-10-20 10:39:00
2016-10-20 11:15:00
2016-10-20 12:03:00
2016-10-20 12:15:00
2016-10-20 12:39:00
2016-10-20 13:39:00
2016-10-20 14:03:00
2016-10-20 14:51:00
2016-10-21 09:15:00
2016-10-21 09:51:00
2016-10-21 11:15:00
2016-10-21 11:39:00
2016-10-21 12:15:00
2016-10-21 13:27:00
2016-10-21 14:27:00
2016-10-21 14:39:00
2016-10-24 09:15:00
2016-10-24 09:27:00
2016-10-24 10:39:00
2016-10-24 11:15:00
2016-10-24 12:39:00
2016-10-24 13:27:00
2016-10-25 09:15:00
2016-10-25 10:15:00
2016-10-25 10:39:00
2016-10-25 11:03:00
2016-10-25 11:27:00
2016-10-25 12:51:00
2016-10-25 13:15:00
2016-10-25 14:39:00
2016-10-25 14:51:00
2016-10-25 15:15:00
2016-10-26 09:27:00
2016-10-26 10:27:00
2016-10-26 10:51:00
2016-10-26 11:51:00
2016-10-26 12:03:00
2016-10-26 12:51:00
2016-10-26 13:15:00
2016-10-26 14:03:00
2016-10-26 14:39:00
2016-10-26 15:03:00
2016-10-27 10:03:00
2016-10-27 10:51:00
2016-10-27 11:15:00
2016-10-27 12:03:00
2016-10-27 12:27:00
2016-10-27 12:51:00
2016-10-27 13:27:00


# STEP 11 which is basically step 10 in FRctal function txt

In [28]:
def step_11(data,n=5,k=2):
    
    min_prev_candles = (n+k)
    
    step_10 = pd.Series(dtype="float64",index=data.index)

    red_op = pd.Series(dtype="float64",index=data.index)
    green_op = pd.Series(dtype="float64",index=data.index)

    for index,row in data.iterrows():

        sub_df = data.loc[:index]

        if row.is_high ==1 :

            sub_df_low = sub_df[sub_df['is_low']==1]
            if sub_df_low.shape[0]>=min_prev_candles:

                last_index = sub_df_low.index[-1]
                row_num = sub_df_low.index.get_loc(last_index)

                end_index = sub_df_low.index[row_num-n]
                start_index = sub_df_low.index[row_num-n-k+1]

                output_val = sub_df_low.loc[start_index:end_index]['Low'].mean()
                red_op.loc[index] = output_val

        elif row.is_low == 1 :

            sub_df_high = sub_df[sub_df['is_high']==1]
            if sub_df_high.shape[0]>=min_prev_candles:

                last_index = sub_df_high.index[-1]
                row_num = sub_df_high.index.get_loc(last_index)

                end_index = sub_df_high.index[row_num-n]
                start_index = sub_df_high.index[row_num-n-k+1]

                output_val = sub_df_high.loc[start_index:end_index]['High'].mean()
                green_op.loc[index] = output_val
                
    data['red_op_continuos'] = red_op
    data['green_op_continuos'] = green_op
    return data
        

In [42]:
data = step_11(data,10,5)

## ALL STEPS IN A SINGLE CELL

In [64]:
data = get_n_trailing_averages_new(data,5)

data['fractal_val_green'] = data.apply(lambda row : get_fractal_val_green(row.is_high,row.High,row.Low),axis=1)
data['fractal_val_red'] = data.apply(lambda row : get_fractal_val_red(row.is_low,row.High,row.Low),axis=1)

data['abs_diff'] = data.apply(lambda row: compute_abs_diff(row.is_low,row.is_high,row.prev_n_op_red_avg,row.prev_n_op_green_avg,row.fractal_val_green,row.fractal_val_red),axis=1)
data['%_growth'] = data.apply(lambda row: compute_percent_growth(row.is_low,row.is_high,row.prev_n_op_red_avg,row.prev_n_op_green_avg,row.fractal_val_green,row.fractal_val_red),axis=1)


In [65]:
neutral_indices = data[(data['is_high']==1) & (data['is_low']==1)].index

green_start = data['prev_n_op_red_avg'].first_valid_index()
red_start = data['prev_n_op_green_avg'].first_valid_index()

if green_start<red_start:
    start_index = green_start
else:
    start_index = red_start


neutral_index_is_high = pd.Series(dtype="float64",index=data.index)

for index in neutral_indices:
    prev_total_data = data.loc[start_index:index]
    prev_normal_data = prev_total_data[~((prev_total_data['is_high']==1) & (prev_total_data['is_low']==1))]

    if not prev_normal_data.empty:
        prev_last_normal_rec = prev_normal_data.iloc[-1]

        if prev_last_normal_rec.is_high:
            data.at[index,'abs_diff'] = data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg']
            data.at[index,'%_growth'] = (data.loc[index]['fractal_val_green']/data.loc[index]['prev_n_op_red_avg'] - 1)*100
            neutral_index_is_high[index] = 1
        else:
            data.at[index,'abs_diff'] = data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg']
            data.at[index,'%_growth'] = (data.loc[index]['fractal_val_red']/data.loc[index]['prev_n_op_green_avg'] - 1)*100
            neutral_index_is_high[index] = 0
    
    
    else:
        if (data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg']) < 0:
            data.at[index,'abs_diff'] = (data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg'])
            data.at[index,'%_growth'] = (data.loc[index]['fractal_val_green']/data.loc[index]['prev_n_op_red_avg'] - 1)*100
            neutral_index_is_high[index] = 1

        elif (data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg']) > 0:
            data.at[index,'abs_diff'] = (data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg'])
            data.at[index,'%_growth'] = (data.loc[index]['fractal_val_red']/data.loc[index]['prev_n_op_green_avg'] - 1)*100
            neutral_index_is_high[index] = 0
        else:
            data.at[index,'abs_diff'] = np.NaN

data = data.assign(neutral_index_is_high = neutral_index_is_high)


In [66]:
data,label_map,op_map = generate_levels(data,'abs_diff')
data = bin_abs_diff(data,label_map,0.5)
data,new_map = step_5_new(data,label_map)
data = step_6(data,new_map)
data = calc_step_7(data,new_map)
data = step_9(data,10,2)
data = step_11(data,10,5)

C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_10508\1052621608.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_A_firsts['shift'] = only_A_firsts[step_7_map[new_map['A']]].shift(step)   #Shifting trailing average
C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_10508\1052621608.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_B_firsts['shift'] = only_B_firsts[step_7_map[new_map['B']]].shift(step)
C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_10508\1052621608.py:13: SettingWithCopyWar

2016-10-21 09:15:00
2016-10-24 13:15:00
2016-10-27 09:15:00
2016-11-01 13:15:00
2016-11-04 13:15:00
2016-11-08 13:15:00
2016-11-09 09:15:00
2016-11-10 09:15:00
2016-11-18 13:15:00
2016-11-21 13:15:00
2016-11-23 09:15:00
2016-11-24 13:15:00
2016-11-29 09:15:00
2016-12-01 09:15:00
2016-12-05 09:15:00
2016-12-07 13:15:00
2016-12-09 09:15:00
2016-12-12 13:15:00
2016-12-14 09:15:00
2016-12-15 09:15:00
2016-12-23 13:15:00
2016-12-26 09:15:00
2016-12-29 09:15:00
2017-01-03 13:15:00
2017-01-06 09:15:00
2017-01-09 09:15:00
2017-01-13 09:15:00
2017-01-13 13:15:00
2017-01-18 09:15:00
2017-01-19 13:15:00
2017-01-23 09:15:00
2017-01-27 09:15:00
2017-02-01 09:15:00
2017-02-02 13:15:00
2017-02-06 09:15:00
2017-02-08 13:15:00
2017-02-13 09:15:00
2017-02-15 13:15:00
2017-02-17 09:15:00
2017-02-23 13:15:00
2017-02-28 13:15:00
2017-03-02 09:15:00
2017-03-03 09:15:00
2017-03-07 09:15:00
2017-03-08 09:15:00
2017-03-10 13:15:00
2017-03-14 09:15:00
2017-03-17 09:15:00
2017-03-22 13:15:00
2017-03-24 13:15:00


In [30]:
def fractal_all(data, trailing_avg_length = 5, bin_threshold = 0.5, trailing_length = 10, trailing_stride = 2, trailing_continuos = 5):

    data = get_n_trailing_averages_new(data,trailing_avg_length)

    data['fractal_val_green'] = data.apply(lambda row : get_fractal_val_green(row.is_high,row.High,row.Low),axis=1)
    data['fractal_val_red'] = data.apply(lambda row : get_fractal_val_red(row.is_low,row.High,row.Low),axis=1)

    data['abs_diff'] = data.apply(lambda row: compute_abs_diff(row.is_low,row.is_high,row.prev_n_op_red_avg,row.prev_n_op_green_avg,row.fractal_val_green,row.fractal_val_red),axis=1)
    data['%_growth'] = data.apply(lambda row: compute_percent_growth(row.is_low,row.is_high,row.prev_n_op_red_avg,row.prev_n_op_green_avg,row.fractal_val_green,row.fractal_val_red),axis=1)

    ## modifying the values for 1,1 fractals
    neutral_indices = data[(data['is_high']==1) & (data['is_low']==1)].index

    green_start = data['prev_n_op_red_avg'].first_valid_index()
    red_start = data['prev_n_op_green_avg'].first_valid_index()

    if green_start<red_start:
        start_index = green_start
    else:
        start_index = red_start


    neutral_index_is_high = pd.Series(dtype="float64",index=data.index)

    for index in neutral_indices:
        prev_total_data = data.loc[start_index:index]
        prev_normal_data = prev_total_data[~((prev_total_data['is_high']==1) & (prev_total_data['is_low']==1))]

        if not prev_normal_data.empty:
            prev_last_normal_rec = prev_normal_data.iloc[-1]

            if prev_last_normal_rec.is_high:
                data.at[index,'abs_diff'] = data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg']
                data.at[index,'%_growth'] = (data.loc[index]['fractal_val_green']/data.loc[index]['prev_n_op_red_avg'] - 1)*100
                neutral_index_is_high[index] = 1
            else:
                data.at[index,'abs_diff'] = data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg']
                data.at[index,'%_growth'] = (data.loc[index]['fractal_val_red']/data.loc[index]['prev_n_op_green_avg'] - 1)*100
                neutral_index_is_high[index] = 0
        
        
        else:
            if (data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg']) < 0:
                data.at[index,'abs_diff'] = (data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg'])
                data.at[index,'%_growth'] = (data.loc[index]['fractal_val_green']/data.loc[index]['prev_n_op_red_avg'] - 1)*100
                neutral_index_is_high[index] = 1

            elif (data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg']) > 0:
                data.at[index,'abs_diff'] = (data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg'])
                data.at[index,'%_growth'] = (data.loc[index]['fractal_val_red']/data.loc[index]['prev_n_op_green_avg'] - 1)*100
                neutral_index_is_high[index] = 0
            else:
                data.at[index,'abs_diff'] = np.NaN

    data = data.assign(neutral_index_is_high = neutral_index_is_high)

    data,label_map,op_map = generate_levels(data,'abs_diff')
    data = bin_abs_diff(data,label_map,bin_threshold)
    data,new_map = step_5_new(data,label_map)
    data = step_6(data,new_map)
    data = calc_step_7(data,new_map)
    data = step_9(data, trailing_length, trailing_stride)
    data = step_11(data, trailing_length, trailing_continuos)
    return data


        


In [67]:
data.to_excel('./fractals_240TF_input_5.xlsx')

In [31]:
fractal_data = fractal_all(data)

C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_16332\1052621608.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_A_firsts['shift'] = only_A_firsts[step_7_map[new_map['A']]].shift(step)   #Shifting trailing average
C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_16332\1052621608.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_B_firsts['shift'] = only_B_firsts[step_7_map[new_map['B']]].shift(step)
C:\Users\TEMP002\AppData\Local\Temp\16\ipykernel_16332\1052621608.py:13: SettingWithCopyWar

2016-10-21 09:15:00
2016-10-24 13:15:00
2016-10-27 09:15:00
2016-11-01 13:15:00
2016-11-04 13:15:00
2016-11-08 13:15:00
2016-11-09 09:15:00
2016-11-10 09:15:00
2016-11-18 13:15:00
2016-11-21 13:15:00
2016-11-23 09:15:00
2016-11-24 13:15:00
2016-11-29 09:15:00
2016-12-01 09:15:00
2016-12-05 09:15:00
2016-12-07 13:15:00
2016-12-09 09:15:00
2016-12-12 13:15:00
2016-12-14 09:15:00
2016-12-15 09:15:00
2016-12-23 13:15:00
2016-12-26 09:15:00
2016-12-29 09:15:00
2017-01-03 13:15:00
2017-01-06 09:15:00
2017-01-09 09:15:00
2017-01-13 09:15:00
2017-01-13 13:15:00
2017-01-18 09:15:00
2017-01-19 13:15:00
2017-01-23 09:15:00
2017-01-27 09:15:00
2017-02-01 09:15:00
2017-02-02 13:15:00
2017-02-06 09:15:00
2017-02-08 13:15:00
2017-02-13 09:15:00
2017-02-15 13:15:00
2017-02-17 09:15:00
2017-02-23 13:15:00
2017-02-28 13:15:00
2017-03-02 09:15:00
2017-03-03 09:15:00
2017-03-07 09:15:00
2017-03-08 09:15:00
2017-03-10 13:15:00
2017-03-14 09:15:00
2017-03-17 09:15:00
2017-03-22 13:15:00
2017-03-24 13:15:00


In [32]:
fractal_data.to_excel('./test_all.xlsx')